# LLM Zoomcamp HW1

In [1]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

## Q1. Running Elastic

In [2]:
es_client = Elasticsearch("http://localhost:9200")

In [3]:
print(es_client.info())

{'name': 'ef7a528df205', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'AWrUfuHvROa1T1LvCEiJbA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## Getting the data

In [4]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

In [6]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

In [7]:
index_name = "course-questions"

In [8]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [10]:
query = "How do I execute a command in a running docker container?"

In [11]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
        }
    },
}

In [12]:
response = es_client.search(index=index_name, body=search_query)

In [13]:
results_doc = []

for hit in response["hits"]["hits"]:
    results_doc.append(hit["_score"])

results_doc

[83.57719, 75.6184, 72.157394, 50.81235, 49.64858]

## Q4. Filtering

In [14]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
            "filter": {"term": {"course": "machine-learning-zoomcamp"}},
        }
    },
}

In [15]:
response4 = es_client.search(index=index_name, body=search_query)

In [16]:
response4["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'I9XHPpABeV-41QPm_1-8',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'QtXIPpABeV-41QPmA18P',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

## Q5. Building a prompt

In [17]:
def build_prompt(query, search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context = ""

    search_results = search_results["hits"]["hits"]

    for doc in search_results:
        context = (
            context
            + context_template.format(
                question=doc["_source"]["question"],
                text=doc["_source"]["text"],
            )
            + "\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
prompt = build_prompt(query, response4)

In [19]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do I execute a command in a running docker container?

    CONTEXT:
    Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or

In [20]:
print(len(prompt))

1490


## Q6. Tokens

In [21]:
import tiktoken

In [22]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [23]:
len(encoding.encode(prompt))

329